In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras import applications
from keras.layers import Dense,GlobalAveragePooling2D
from keras.models import Model

import numpy as np
import matplotlib.pyplot as plt
import os 
import cv2

Using TensorFlow backend.
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
DATADIR = "train_images"
CATEGORIES = [str(i) for i in range(121)]

In [3]:
# IMG_SIZE = 95


# plt.imshow(new_array, cmap = "gray")

NameError: name 'new_array' is not defined

In [4]:
IMG_SIZE = 95

training_data = []

def create_training_data():
    for index, category in enumerate(CATEGORIES,0):
        path = os.path.join(DATADIR, category) 
        class_num = index
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img))
            new_array = cv2.resize(img_array,(IMG_SIZE, IMG_SIZE))
            training_data.append([new_array, class_num])
            
create_training_data()
        

In [5]:
import random 

random.shuffle(training_data)

In [6]:

X = []
y = []

class_weight = {}

for features, label in training_data:
    
    X.append(features)
    
    one_hot = [0 for i in range(121)]
    one_hot[label] = 1
    
    y.append(one_hot)
    
    if label in class_weight:
      class_weight[label] += 1.0
      
    else:
      class_weight[label] = 1.0
      
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3) 
y = np.array(y)

# create class weights
for class_num in class_weight:
  class_weight[class_num] = 1 - (class_weight[class_num] / len(y))
  

In [7]:
X.shape

(24204, 95, 95, 3)

In [8]:
X = X/255.0 # normalize

In [2]:
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (95,95,3))

In [4]:
x = model.output
x = Flatten()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(121, activation="softmax")(x)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
model=Model(inputs=base_model.input,outputs=predictions)

NameError: name 'base_model' is not defined

In [7]:
for layer in model.layers:
    print(layer.name)

input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_conv4
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_conv4
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_conv4
block5_pool


In [ ]:
model.compile(loss = "categorical_crossentropy",
             optimizer = "adam",
             metrics = ["accuracy"])

In [67]:
# model = Sequential()
# N_Classes = 121

# # model.add(InputLayer())
# model.add(Conv2D(64,(3,3),padding = "same", input_shape = X.shape[1:]))
# model.add(Activation("relu"))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(64, (3,3), padding = "same"))
# model.add(Activation("relu"))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(64, (3,3), padding = "same"))
# model.add(Activation("relu"))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(100))
# model.add(Dense(N_Classes, activation='softmax'))

# model.compile(loss = "categorical_crossentropy",
#              optimizer = "adam",
#              metrics = ["accuracy"])




In [ ]:
model.fit(X, y, batch_size = 64, epochs = 500, validation_split = 0.1)

In [0]:
from keras.preprocessing.image import ImageDataGenerator 

train_generator = ImageDataGenerator(
    
    rotation_range = 360,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest'

)

val_generator = ImageDataGenerator(
    
    horizontal_flip = True,
    vertical_flip = True,

)

train_generator.fit(X) # computes quantities required for featurewise normalization

N_epochs = 3
Batch_size = 64

# fits the model on batches with real-time data augmentation:
model.fit_generator(train_generator.flow(X, y, batch_size = Batch_size),
                    steps_per_epoch=len(X) / Batch_size, epochs = N_epochs)
    

Epoch 1/3
379/378 [==============================] - 30s 79ms/step - loss: 2.9305 - acc: 0.2712
Epoch 2/3
379/378 [==============================] - 31s 83ms/step - loss: 2.7652 - acc: 0.2984
Epoch 3/3
379/378 [==============================] - 30s 80ms/step - loss: 2.7195 - acc: 0.3060


In [0]:
model.fit_generator(train_generator.flow(X, y, batch_size = Batch_size),
                    steps_per_epoch=len(X) / Batch_size, epochs = N_epochs)
    

Epoch 1/3
379/378 [==============================] - 31s 81ms/step - loss: 3.2255 - acc: 0.2306
Epoch 2/3
379/378 [==============================] - 31s 81ms/step - loss: 2.8592 - acc: 0.2826
Epoch 3/3
379/378 [==============================] - 31s 81ms/step - loss: 2.7500 - acc: 0.3025


# Test and Write

In [0]:
from PIL import Image

directory_in_str = "drive/My Drive/AML_plankton_project/test_images/test"


results = list()


for file in os.listdir(directory_in_str):
    if file.endswith(".jpg"):
        
        filename = os.fsdecode(file)
        image_path = os.path.join(directory_in_str, filename)
        
        img_array = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        new_array = cv2.resize(img_array,(IMG_SIZE, IMG_SIZE))
        norm_array = new_array / 255
        norm_array = np.array(norm_array).reshape(-1,IMG_SIZE, IMG_SIZE, 1) 
        prediction = model.predict_classes(norm_array)
        
        results.append([filename, prediction[0]])
        
        

# print(results)            
test_results = pd.DataFrame(results, columns= ["image","class"])

print(test_results)


In [0]:
test_results.to_csv('output_eerste_keras.csv', index=False)

In [0]:
from google.colab import files

files.download("output_eerste_keras.csv")